In [ ]:
import os, shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from PIL import Image
from keras.utils import plot_model
from IPython.display import Image 

# preprocessing
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# modelling
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras import utils
from keras import optimizers
from keras.models import Sequential
from keras.losses import binary_crossentropy, sparse_categorical_crossentropy
from keras.optimizers import SGD, Adam
from keras.applications.vgg19 import VGG19
from keras import losses
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
import math 

# callbacks
from keras.callbacks import History

In [ ]:
IMAGE_PATH = "siim-isic-melanoma-classification/jpeg"

train_folder = os.path.join(IMAGE_PATH, 'train')
test_folder = os.path.join(IMAGE_PATH, 'test')
# val_folder = os.path.join(IMAGE_PATH, 'val')

# train_df = pd.read_csv('siim-isic-melanoma-classification/train.csv')
# test_df = pd.read_csv('siim-isic-melanoma-classification/test.csv')

# #Training data
# print('Training data shape: ', train_df.shape)
# train_df.head(5)

In [ ]:
# train_df.groupby(['benign_malignant']).count()['sex'].to_frame()

In [ ]:
# train_df.info()

In [ ]:
#test_df.info()

In [ ]:
# images_df = train_df[['image_name', 'benign_malignant']]

# images_df.head()

In [ ]:
# benign = images_df[images_df['benign_malignant']=='benign']
# malignant = images_df[images_df['benign_malignant']=='malignant']

In [ ]:
#benign['image_name'] = benign['image_name'].astype(str) + '.jpg'
#malignant['image_name'] = malignant['image_name'].astype(str) + '.jpg'

In [ ]:
#images = malignant['image_name'].values

# benign_dir = train_folder+'/benign'

In [ ]:
#for i in images:
#    i = os.path.join(train_folder, i)
#    os.remove(i)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

# get all the data in the directory /train and reshape them
train_generator = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(224, 224),  
        batch_size=3479,
        classes=['benign', 'malignant'])

# get all the data in the directory /train and reshape them
test_generator = train_datagen.flow_from_directory(
        test_folder, 
        target_size=(224, 224), 
        # batch_size = 874,
        classes=['benign', 'malignant']) 

# get all the data in the directory /val and reshape them
val_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=(224, 224), 
        class_mode='binary',
        subset='validation')




In [ ]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [ ]:

#preview training image
array_to_img(train_images[145])


In [ ]:
# get shape of images in train and test

print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(test_images))
print(np.shape(test_labels))

In [ ]:

# reshape images to contain dimensions into a single vector
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

In [ ]:
# reshape images to contain dimensions into a single vector
train_y = np.reshape(train_labels[:,0], (3377,1))
test_y = np.reshape(test_labels[:,0], (1073,1))
val_y = np.reshape(val_labels[:,0], (873,1))

In [ ]:
#preview label to identify class valujes
train_labels

In [ ]:

#identify the categorical values of 0 and 1
train_generator.class_indices

In [ ]:
train_labels_final = train_labels.T[[1]]
np.shape(train_labels_final)


In [ ]:
test_labels_final = test_labels.T[[1]]
np.shape(test_labels_final)

In [ ]:
array_to_img(train_images[210])

In [ ]:
train_labels_final[:,210]

### Base CNN Model

In [ ]:
np.random.seed(123)

# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
 
# model.add(layers.Flatten())
# model.add(layers.Dense(10, activation='softmax'))


base_model = models.Sequential()
base_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(256 ,256,  3)))
base_model.add(layers.MaxPooling2D((2, 2)))

base_model.add(layers.Conv2D(32, (4, 4), activation='relu'))
base_model.add(layers.MaxPooling2D((2, 2)))

base_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
base_model.add(layers.MaxPooling2D((2, 2)))

base_model.add(layers.Flatten())
base_model.add(layers.Dense(64, activation='relu'))
base_model.add(layers.Dense(1, activation='sigmoid'))

base_model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
 
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer='sgd',
#               metrics=['accuracy'])

base_model.summary()

In [ ]:
plot_model(base_model, show_shapes=True, show_layer_names=True)

In [ ]:
history = base_model.fit(train_images,
          train_y,
          epochs=10,
          batch_size=128,
          validation_data=(val_images, val_y))

In [ ]:
results_train = base_model.evaluate(train_images, train_y)

In [ ]:
results_test = base_model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
visualize_training_results(history)

### Model 2

In [ ]:
np.random.seed(123)

base_model = models.Sequential()
high_base_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(256 ,256,  3)))
high_base_model.add(layers.MaxPooling2D((2, 2)))

high_base_model.add(layers.Conv2D(32, (4, 4), activation='relu'))
high_base_model.add(layers.MaxPooling2D((2, 2)))

high_base_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
high_base_model.add(layers.MaxPooling2D((2, 2)))

high_base_model.add(layers.Flatten())
high_base_model.add(layers.Dense(64, activation='relu'))
high_base_model.add(layers.Dense(1, activation='sigmoid'))

high_base_model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy']) 

high_base_model.summary()

In [ ]:
plot_model(high_base_model, show_shapes=True, show_layer_names=True)

In [ ]:
history = high_base_model.fit(train_images,
          train_y,
          epochs=5,
          batch_size=64,
          validation_data=(val_images, val_y))

In [ ]:
results_train = high_base_model.evaluate(train_images, train_y)

In [ ]:
results_test = high_base_model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
visualize_training_results(history)